In [1]:
# Load packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.stats

In [2]:
# Load shots
with open('./data/players_shots_clean.pickle', 'rb') as f:
    players_shots = pickle.load(f)
with open('./data/player_position.pickle', 'rb') as f:
    players_position = pickle.load(f)


In [3]:
# VARIABLES
X_MIN, X_MAX = (-250, 250) 
Y_MIN, Y_MAX = (0, 422.5)

XX, YY = np.mgrid[0:1:51j, 0:1:51j]
POS = np.vstack([XX.ravel(), YY.ravel()])

In [4]:
# Functions
def estimate_density(df):
    X = df.LOC_X.to_numpy(dtype='float')
    Y = df.LOC_Y.to_numpy(dtype='float')
    
    # Rescaling to [0, 1]
    X = (X - X_MIN) / (X_MAX - X_MIN)
    Y = (Y - Y_MIN) / (Y_MAX - Y_MIN)
    
    # Kernel density estimates
    values = np.vstack([X, Y])
    kernel = scipy.stats.gaussian_kde(values, bw_method='silverman')
    density = kernel(POS).T
    density[density < 1e-16] = 0
    return np.reshape(density, XX.shape)

In [7]:
players_position['POSITION'].unique()

array(['Forward', 'Center', 'Center-Forward', 'Guard', 'Forward-Center',
       'Forward-Guard', 'Guard-Forward'], dtype=object)

In [10]:
# Remove Center player
# They shots only under the basket...
not_centers = players_position.query(
    "POSITION != 'Center' & POSITION != 'Center-Forward' & POSITION != 'Forward-Center'"
)
players_shots = players_shots.loc[[i in not_centers.PLAYER_ID.values for i in players_shots.PLAYER_ID.values]]
#players_shots = players_shots.query("PLAYER_NAME != 'Mitchell Robinson'")

In [14]:
shots_density = players_shots\
    .groupby('PLAYER_ID')\
    .apply(estimate_density)

In [15]:
shots_density_df = players_shots[['PLAYER_ID', 'PLAYER_NAME']]\
    .drop_duplicates()\
    .join(pd.DataFrame({'DENSITY': shots_density}), on='PLAYER_ID')\
    .reset_index()\
    .drop('index', axis=1)

In [17]:
# Save dataframe
with open('./data/players_shots_density.pickle', 'wb') as f:
    pickle.dump(shots_density_df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
# Now, we do the same for the made shots.
players_shots_made = players_shots[players_shots.SHOT_MADE_FLAG == 1]

In [19]:
shots_density_made = players_shots_made\
    .groupby('PLAYER_ID')\
    .apply(estimate_density)

In [20]:
shots_density_made_df = players_shots[['PLAYER_ID', 'PLAYER_NAME']]\
    .drop_duplicates()\
    .join(pd.DataFrame({'DENSITY': shots_density_made}), on='PLAYER_ID')\
    .reset_index()\
    .drop('index', axis=1) 

In [21]:
# Save dataframe
with open('./data/players_shots_density_made.pickle', 'wb') as f:
    pickle.dump(shots_density_made_df, f, protocol=pickle.HIGHEST_PROTOCOL)